<a href="https://colab.research.google.com/github/ykykyk0814/GUI_course/blob/main/DS301_Project_midway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tqdm
import glob

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import rgb2gray

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import to_categorical
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
import keras.backend as K
from tensorflow.keras.backend import sum as k_sum, round as k_round, clip as k_clip, epsilon as k_epsilon

from typeguard import typechecked
from typing import Optional

In [32]:
!gdown https://drive.google.com/uc?id=10-b4PKd6UUTkZU3SdOn_hbgZHU8CUjT_
import zipfile

with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

Downloading...
From (original): https://drive.google.com/uc?id=10-b4PKd6UUTkZU3SdOn_hbgZHU8CUjT_
From (redirected): https://drive.google.com/uc?id=10-b4PKd6UUTkZU3SdOn_hbgZHU8CUjT_&confirm=t&uuid=e69232b7-f885-4c03-be3c-aee07f1bda10
To: /content/archive.zip
100% 35.8M/35.8M [00:00<00:00, 190MB/s]


In [33]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

train_dataset  = train_datagen.flow_from_directory(directory = 'Alzheimer_s Dataset/train/',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 32)

valid_dataset = valid_datagen.flow_from_directory(directory = 'Alzheimer_s Dataset/train/',
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 32)

test_dataset = test_datagen.flow_from_directory(directory = 'Alzheimer_s Dataset/test/',
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 32)

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [34]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.backend import sum as k_sum, round as k_round, clip as k_clip, epsilon as k_epsilon

# Set global variables
compression = 0.5
num_classes = 4
l = 4

# Dense block
def denseblock(input, num_filter=12, dropout_rate=0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter * compression), (3, 3), use_bias=False, padding='same')(relu)
        if dropout_rate > 0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        temp = layers.Concatenate(axis=-1)([temp, Conv2D_3_3])
    return temp

# Transition block
def transition(input, num_filter=12, dropout_rate=0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter * compression), (1, 1), use_bias=False, padding='same')(relu)
    if dropout_rate > 0:
        Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    return layers.AveragePooling2D(pool_size=(2, 2))(Conv2D_BottleNeck)

# Output layer
def output_layer(input):
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.GlobalAveragePooling2D()(relu)
    return layers.Dense(num_classes, activation='softmax')(AvgPooling)

# F1 score custom metric with debugging
def f1_score(y_true, y_pred):
    true_positives = k_sum(k_round(k_clip(y_true * y_pred, 0, 1)))
    possible_positives = k_sum(k_round(k_clip(y_true, 0, 1)))
    predicted_positives = k_sum(k_round(k_clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + k_epsilon())
    recall = true_positives / (possible_positives + k_epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + k_epsilon())

    # Debugging statement
    tf.print("F1 Score:", f1)

    return f1

# Learning rate scheduler callback with debugging
def scheduler(epoch, lr):
    new_lr = lr * tf.math.exp(-0.1) if epoch >= 10 else lr
    tf.print("Learning Rate for epoch", epoch, ":", new_lr)
    return new_lr

lr_scheduler = LearningRateScheduler(scheduler)

# Create the combined model
def create_combined_model():
    base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    base_model_output = base_model.output
    dense_block_output = denseblock(base_model_output, num_filter=32, dropout_rate=0.2)
    transition_block_output = transition(dense_block_output, num_filter=32, dropout_rate=0.2)
    output = output_layer(transition_block_output)

    model = Model(inputs=base_model.input, outputs=output)
    return model

# Instantiate and compile the model
model = create_combined_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score])

# Train the model
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=17,
    verbose=1,
    callbacks=[lr_scheduler]
)

model.summary()


Learning Rate for epoch 0 : 0.0010000000474974513
Epoch 1/17
F1 Score: 0
  1/129 ━━━━━━━━━━━━━━━━━━━━ 51:59 24s/step - accuracy: 0.2188 - f1_score: 0.0000e+00 - loss: 1.4535F1 Score: 0
  2/129 ━━━━━━━━━━━━━━━━━━━━ 29:25 14s/step - accuracy: 0.1797 - f1_score: 0.0000e+00 - loss: 1.4716F1 Score: 0
  3/129 ━━━━━━━━━━━━━━━━━━━━ 24:20 12s/step - accuracy: 0.1788 - f1_score: 0.0000e+00 - loss: 1.4672F1 Score: 0
  4/129 ━━━━━━━━━━━━━━━━━━━━ 20:40 10s/step - accuracy: 0.1868 - f1_score: 0.0000e+00 - loss: 1.4628F1 Score: 0
  5/129 ━━━━━━━━━━━━━━━━━━━━ 19:46 10s/step - accuracy: 0.1932 - f1_score: 0.0000e+00 - loss: 1.4595F1 Score: 0
  6/129 ━━━━━━━━━━━━━━━━━━━━ 18:22 9s/step - accuracy: 0.2001 - f1_score: 0.0000e+00 - loss: 1.4568 F1 Score: 0
  7/129 ━━━━━━━━━━━━━━━━━━━━ 17:59 9s/step - accuracy: 0.2085 - f1_score: 0.0000e+00 - loss: 1.4522F1 Score: 0
  8/129 ━━━━━━━━━━━━━━━━━━━━ 17:06 8s/step - accuracy: 0.2147 - f1_score: 0.0000e+00 - loss: 1.4485F1 Score: 0.0588235147
  9/129 ━━━━━━━━━━━━━━

In [37]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [38]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20) # when i run it for 50 epochs

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:

def Train_Val_Plot(acc,val_acc,loss,val_loss,auc,val_auc,precision,val_precision,f1,val_f1):

    fig, (ax1, ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize= (20,5))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('History of Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('History of Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['training', 'validation'])

    ax3.plot(range(1, len(auc) + 1), auc)
    ax3.plot(range(1, len(val_auc) + 1), val_auc)
    ax3.set_title('History of AUC')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('AUC')
    ax3.legend(['training', 'validation'])

    ax4.plot(range(1, len(precision) + 1), precision)
    ax4.plot(range(1, len(val_precision) + 1), val_precision)
    ax4.set_title('History of Precision')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Precision')
    ax4.legend(['training', 'validation'])

    ax5.plot(range(1, len(f1) + 1), f1)
    ax5.plot(range(1, len(val_f1) + 1), val_f1)
    ax5.set_title('History of F1-score')
    ax5.set_xlabel('Epochs')
    ax5.set_ylabel('F1 score')
    ax5.legend(['training', 'validation'])


    plt.show()


Train_Val_Plot(history.history['accuracy'],history.history['val_accuracy'],
               history.history['loss'],history.history['val_loss'],
               history.history['auc'],history.history['val_auc'],
               history.history['precision'],history.history['val_precision'],
               history.history['f1_score'],history.history['val_f1_score']
              )

In [ ]:
scores = model.evaluate_generator(test_dataset)

In [ ]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

In [ ]:
kpred = model.predict(test_dataset)

In [ ]:
pred = kpred.tolist()
predictions = []
for i in pred:
  predictions.append(i.index(max(i)))

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
print(classification_report(test_dataset.classes,predictions))